# Project submission
**Due Friday May 16th before class.** Counts for 25% of the final course grade.

You should address all the questions relevant to your project.
You will not be graded based on the values of the model performance, but on whether or not you have applied the right methodology: formulated the business model, translated it into a right machine learning approach, analyzed your data, prepared it for modeling, applied at least 5 different machine learning algorithms, as well as neural networks, used cross validation for model tuning, justified your tuning metric, set up the proper machine learning pipeline without data leakage, evaluated your model using all the relevant metrics, interpreted your model and justified all your decisions.

If you have tried different approaches, please include them all, and not just the best one.
If doing some feature engineering has improved your model, also please include all of the steps, not just the most successful ones.

You should submit the notebook with the code, output and explanations. The notebook should be executable and comprehensible.

The points will be deducted for the following reasons:
- data leakage
- unjustified decisions (no discussion on: choice of metric for optimization, blind removal of features, blind removal of outliers...)
- notebook not comprehensible
- notebook with incomplete output
- notebook not executable
- blind copy pasting from ChatGPT, if the copied code is not suitable for the task
- writing your own code (or copy pasting them from outside source) for simple functions that we covered and that already exist in `sklearn` (train test split, plain grid search, encoding of categorical variables,...), as this leads to:
    - convoluted code prone to bugs
    - code that is hard to understand and review
    - waste of data scientist's time if ready-made simple functions exist

Additional points will be awarded for trying and testing different relevant approaches, from exploratory data analysis, to feature engineering, to modeling and evaluation.

There should be one submission per group, but team member evaluation can be submitted per person. If not submitted, the default is that all the team members have contributed equally to the project and should get the same grade.

### Group number:
### Student IDs:
### Project name:

## What business problem are you solving?
- Please state clearly what business problem are you solving. (one sentence)
- Elaborate why is this a relevant problem, and what can you do with the model output to create business value, i.e., how is the model output actionable. (2-3 paragraphs)

## What is the machine learning problem that you are solving?
- Please state clearly what is the ML problem. 
- If applicable state your target.

## Data exploration and preparation 

- How many data instances do you have?
- Do you have duplicates?
- How many features? What type are they?
- If they are categorical, what categories they have, what is their frequency?
- If they are numerical, what is their distribution?
- Do you have outliers, and do you need to do anything about them?
- What is the distribution of the target variable?
- If you have a target, you can also check the relationship between the target and the variables.
- Do you have missing data? If yes, how are you going to handle it?
- Can you use the features in their original form, or do you need to alter them in some way?
- What have you learned about your data? Is there anything that can help you in feature engineering or modeling?


## Feature engineering
Creating good features is probably the most important step in the machine learning process. 
This might involve doing:
- transformations
- aggregating over data points or over time and space, or finding differences (for example: differences between two monthly bills, time difference between two contacts with the client) 
- creating dummy (binary) variables
- discretization

Business insight is very relevant in this process. If it is possible you can also find additional relevant data.

## Modeling
You should implement AT LEAST FIVE approaches we covered, and tune of at least two hyperparameters of each approach.
Do not forget that you should split your data.
You should do model selection and tuning using cross validation on the train set, avoiding data leakage.
Explain and justify what is the metric you are using for model selection and tuning. If your data is imbalanced, consider using techniques for data balancing.

Separately, you should train a neural network. Visualize the training and validation loss. Discuss the network performance

In model selection, make sure when you compare different models and approaches that you compare them on the same dataset, though different transformations could be applied to the comparison dataset.

## Model evaluation

After selecting your final model, which could be a compromise of performance, interpretability and complexity, you should evaluate its performance on the test set. 
You might have tuned your model using a certain metric, but now you should describe the model performance using all relevant metrics. 
If you have some business insight, why a certain metric is relevant, you should explain it. 
Construct a suitable baseline to benchmark your result and to put them in the context.
Discuss your results, do they seem good enough to be used in practice? If not, what should be improved. Discuss what type of errors is your model making.


## Model interpretation

Use at least two different techniques for model interpretability. Discuss what are the most important features of your model, and how they impact the model performance. Pick a few examples of errors that your model is making, and check which features lead to thess errors.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


In [4]:
df = pd.read_csv('pr13_stocks (1).csv', index_col=0)
df = df.sample(frac=0.05, random_state=42)  # frac=0.1 means 5% of the data

df.head()

,Date,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Volume,Open,High,Low,Close
75721,2008-12-11 00:00:00-05:00,0.0,0.0,nintendo,NTDOY,gaming,japan,2476000.0,9.550000,9.550000,9.130000,9.172000
80184,2022-08-31 00:00:00-04:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,4054100.0,77.253671,77.565610,76.171632,76.239868
19864,2008-02-25 00:00:00-05:00,0.0,0.0,american express,AXP,finance,usa,8155900.0,35.124497,35.373717,34.189923,35.210167
76699,2020-08-27 00:00:00-04:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,2208900.0,73.579890,74.036335,73.430853,73.645096
92991,2014-03-21 00:00:00-04:00,0.0,0.0,the walt disney company,DIS,entertainment,usa,7840900.0,75.429113,75.511945,73.671312,73.947403


In [5]:
df['Date'] = pd.to_datetime(df['Date'], utc=True)

df

,Date,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Volume,Open,High,Low,Close
75721,2008-12-11 05:00:00+00:00,0.0,0.0,nintendo,NTDOY,gaming,japan,2476000.0,9.550000,9.550000,9.130000,9.172000
80184,2022-08-31 04:00:00+00:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,4054100.0,77.253671,77.565610,76.171632,76.239868
19864,2008-02-25 05:00:00+00:00,0.0,0.0,american express,AXP,finance,usa,8155900.0,35.124497,35.373717,34.189923,35.210167
76699,2020-08-27 04:00:00+00:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,2208900.0,73.579890,74.036335,73.430853,73.645096
92991,2014-03-21 04:00:00+00:00,0.0,0.0,the walt disney company,DIS,entertainment,usa,7840900.0,75.429113,75.511945,73.671312,73.947403
...,...,...,...,...,...,...,...,...,...,...,...,...
44719,2019-09-03 04:00:00+00:00,0.0,0.0,southwest airlines,LUV,aviation,usa,4298000.0,50.427139,50.767274,49.474767,49.931519
20980,2015-06-09 04:00:00+00:00,0.0,0.0,costco,COST,retail,usa,2039100.0,119.588688,120.317038,119.493307,119.692734
57224,2021-01-21 05:00:00+00:00,0.0,0.0,salesforce / slack,CRM,technology,usa,6785900.0,223.690002,224.740005,220.820007,222.169998
23910,2020-04-22 04:00:00+00:00,0.0,0.0,cisco,CSCO,technology,usa,18330700.0,37.348931,38.091022,37.204132,37.792374


In [6]:
df.isna().sum()

Date             2
Dividends        4
Stock Splits    33
Brand_Name      10
Ticker          16
Industry_Tag     0
Country          3
Volume          34
Open            39
High            35
Low             42
Close           10
dtype: int64

In [ ]:
#drop the 16 rows without any date data
df = df.dropna(subset=['Date'])

In [8]:
df

,Date,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Volume,Open,High,Low,Close
75721,2008-12-11 05:00:00+00:00,0.0,0.0,nintendo,NTDOY,gaming,japan,2476000.0,9.550000,9.550000,9.130000,9.172000
80184,2022-08-31 04:00:00+00:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,4054100.0,77.253671,77.565610,76.171632,76.239868
19864,2008-02-25 05:00:00+00:00,0.0,0.0,american express,AXP,finance,usa,8155900.0,35.124497,35.373717,34.189923,35.210167
76699,2020-08-27 04:00:00+00:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,2208900.0,73.579890,74.036335,73.430853,73.645096
92991,2014-03-21 04:00:00+00:00,0.0,0.0,the walt disney company,DIS,entertainment,usa,7840900.0,75.429113,75.511945,73.671312,73.947403
...,...,...,...,...,...,...,...,...,...,...,...,...
44719,2019-09-03 04:00:00+00:00,0.0,0.0,southwest airlines,LUV,aviation,usa,4298000.0,50.427139,50.767274,49.474767,49.931519
20980,2015-06-09 04:00:00+00:00,0.0,0.0,costco,COST,retail,usa,2039100.0,119.588688,120.317038,119.493307,119.692734
57224,2021-01-21 05:00:00+00:00,0.0,0.0,salesforce / slack,CRM,technology,usa,6785900.0,223.690002,224.740005,220.820007,222.169998
23910,2020-04-22 04:00:00+00:00,0.0,0.0,cisco,CSCO,technology,usa,18330700.0,37.348931,38.091022,37.204132,37.792374


In [9]:
df[df['Brand_Name'].isna() & df['Ticker'].isna()]

,Date,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Volume,Open,High,Low,Close


In [10]:
#drop the row without both Brand_name and Ticker as we cannot say which company this entry refers to
df = df[~(df['Brand_Name'].isna() & df['Ticker'].isna())].copy()

In [11]:
df

,Date,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Volume,Open,High,Low,Close
75721,2008-12-11 05:00:00+00:00,0.0,0.0,nintendo,NTDOY,gaming,japan,2476000.0,9.550000,9.550000,9.130000,9.172000
80184,2022-08-31 04:00:00+00:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,4054100.0,77.253671,77.565610,76.171632,76.239868
19864,2008-02-25 05:00:00+00:00,0.0,0.0,american express,AXP,finance,usa,8155900.0,35.124497,35.373717,34.189923,35.210167
76699,2020-08-27 04:00:00+00:00,0.0,0.0,colgate palmolive,CL,consumer goods,usa,2208900.0,73.579890,74.036335,73.430853,73.645096
92991,2014-03-21 04:00:00+00:00,0.0,0.0,the walt disney company,DIS,entertainment,usa,7840900.0,75.429113,75.511945,73.671312,73.947403
...,...,...,...,...,...,...,...,...,...,...,...,...
44719,2019-09-03 04:00:00+00:00,0.0,0.0,southwest airlines,LUV,aviation,usa,4298000.0,50.427139,50.767274,49.474767,49.931519
20980,2015-06-09 04:00:00+00:00,0.0,0.0,costco,COST,retail,usa,2039100.0,119.588688,120.317038,119.493307,119.692734
57224,2021-01-21 05:00:00+00:00,0.0,0.0,salesforce / slack,CRM,technology,usa,6785900.0,223.690002,224.740005,220.820007,222.169998
23910,2020-04-22 04:00:00+00:00,0.0,0.0,cisco,CSCO,technology,usa,18330700.0,37.348931,38.091022,37.204132,37.792374


In [12]:
df['Brand_Name'] = df['Brand_Name'].str.lower()
df['Ticker'] = df['Ticker'].str.upper()

brand_to_ticker = df.dropna(subset=['Brand_Name', 'Ticker'])\
                        .drop_duplicates(subset=['Brand_Name'])\
                        .set_index('Brand_Name')['Ticker'].to_dict()

ticker_to_brand = df.dropna(subset=['Brand_Name', 'Ticker'])\
                        .drop_duplicates(subset=['Ticker'])\
                        .set_index('Ticker')['Brand_Name'].to_dict()

    # Fill missing Ticker using Brand_Name
df.loc[df['Ticker'].isna() & df['Brand_Name'].notna(), 'Ticker'] = (
        df.loc[df['Ticker'].isna() & df['Brand_Name'].notna(), 'Brand_Name']
        .map(brand_to_ticker)
    )

    # Fill missing Brand_Name using Ticker
df.loc[df['Brand_Name'].isna() & df['Ticker'].notna(), 'Brand_Name'] = (
    df.loc[df['Brand_Name'].isna() & df['Ticker'].notna(), 'Ticker']
    .map(ticker_to_brand))

In [13]:
df.isna().sum() 

Date             0
Dividends        4
Stock Splits    33
Brand_Name       0
Ticker           0
Industry_Tag     0
Country          3
Volume          34
Open            39
High            35
Low             42
Close           10
dtype: int64

In [14]:
#As dividend payments and stock splits are sparse, event-based features, missing values here are likely because 
#no such event has occured. Therefore, missing values are filled with 0. Also, the occurence of missing values 
#is relatively low here(59 and 458, respectively) compared to the overall dataset size(~100,000)
df['Dividends'] = df['Dividends'].fillna(0.0)
df['Stock Splits'] = df['Stock Splits'].fillna(0.0)

In [15]:
df.isna().sum()

Date             0
Dividends        0
Stock Splits     0
Brand_Name       0
Ticker           0
Industry_Tag     0
Country          3
Volume          34
Open            39
High            35
Low             42
Close           10
dtype: int64

In [16]:
df['Industry_Tag'].value_counts()

Industry_Tag
technology            1048
retail                 429
automotive             401
finance                359
apparel                355
consumer goods         346
food & beverage        315
entertainment          212
aviation               196
gaming                 184
footwear               180
food                   168
hospitality            158
e-commerce             140
healthcare             116
manufacturing          111
logistics               97
luxury goods            72
financial services      35
music                   28
social media            19
fitness                 17
cryptocurrency          12
Name: count, dtype: int64

In [17]:
#Imputing industry tag values with the most frequent value for each brand

df['Industry_Tag'] = df['Industry_Tag'].str.strip().str.lower()
industry_map = (
        df.dropna(subset=['Industry_Tag'])
          .groupby('Brand_Name')['Industry_Tag']
          .agg(lambda x: x.mode()[0]) 
          .to_dict()
    )

df['Industry_Tag'] = df.apply(
        lambda row: industry_map[row['Brand_Name']]
        if pd.isna(row['Industry_Tag']) or row['Industry_Tag'] != industry_map.get(row['Brand_Name'])
        else row['Industry_Tag'],
        axis=1
    )

In [18]:
df['Industry_Tag'].value_counts()

Industry_Tag
technology            1048
retail                 429
automotive             401
finance                359
apparel                355
consumer goods         346
food & beverage        315
entertainment          212
aviation               196
gaming                 184
footwear               180
food                   168
hospitality            158
e-commerce             140
healthcare             116
manufacturing          111
logistics               97
luxury goods            72
financial services      35
music                   28
social media            19
fitness                 17
cryptocurrency          12
Name: count, dtype: int64

In [19]:
#same for country

df['Country'] = df['Country'].str.strip()

country_map = (
        df.dropna(subset=['Country'])
          .groupby('Brand_Name')['Country']
          .agg(lambda x: x.mode()[0]) 
          .to_dict()
    )

df['Country'] = df.apply(
        lambda row: country_map[row['Brand_Name']]
        if pd.isna(row['Country']) or row['Country'] != country_map.get(row['Brand_Name'])
        else row['Country'],
        axis=1
    )

In [20]:
print(df['Country'].value_counts())
print(df.isna().sum())
print(f"Number of duplicates: {df.duplicated().sum()}")

Country
usa            3938
japan           302
germany         273
netherlands     202
france          148
switzerland     105
canada           30
Name: count, dtype: int64
Date             0
Dividends        0
Stock Splits     0
Brand_Name       0
Ticker           0
Industry_Tag     0
Country          0
Volume          34
Open            39
High            35
Low             42
Close           10
dtype: int64
Number of duplicates: 0


In [21]:
print(df.shape)
df = df.drop(columns=['Open', 'High', 'Low','Brand_Name'])
df = df.dropna(subset=['Close'])
print(df.shape)

(4998, 12)
(4988, 8)


Delete Open, High and Low since this data is same day date and not lagged --> otherwise data leakage
Delete Brand_Name because it contains the same info as Ticker
Delete 194 with no Data since it is only 194/100k values and is spread between dates/companies/strategies

In [22]:
df['Date'] = pd.to_datetime(df['Date']) ### was machen wir mit den Stunden???

# Time-based features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek  # 0 = Monday, 6 = Sunday

# Running day number
df['Day_Number'] = (df['Date'] - df['Date'].min()).dt.days
df = df.sort_values(by='Date').reset_index(drop=True)
df

,Date,Dividends,Stock Splits,Ticker,Industry_Tag,Country,Volume,Close,Year,Month,Day,DayOfWeek,Day_Number
0,2000-01-04 05:00:00+00:00,0.0,0.0,PHG,technology,netherlands,1646000.0,17.986570,2000,1,4,1,0
1,2000-01-10 05:00:00+00:00,0.0,0.0,UL,consumer goods,netherlands,190440.0,7.969457,2000,1,10,0,6
2,2000-01-11 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,441548800.0,0.702910,2000,1,11,1,7
3,2000-01-13 05:00:00+00:00,0.0,0.0,CL,consumer goods,usa,2643000.0,19.505140,2000,1,13,3,9
4,2000-01-18 05:00:00+00:00,0.0,0.0,PG,consumer goods,usa,4456800.0,31.210438,2000,1,18,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,2023-09-14 04:00:00+00:00,0.0,0.0,V,finance,usa,12000000.0,241.500000,2023,9,14,3,8653
4984,2023-09-15 04:00:00+00:00,0.0,0.0,UBER,technology,usa,16125700.0,47.520000,2023,9,15,4,8654
4985,2023-09-15 04:00:00+00:00,0.0,0.0,TSLA,automotive,usa,133422800.0,274.390015,2023,9,15,4,8654
4986,2023-09-15 04:00:00+00:00,0.0,0.0,LVMUY,luxury goods,france,486695.0,160.880005,2023,9,15,4,8654


In [23]:
def add_lags(df, lags):
    for lag in lags:
        df[f'Close_Lag_{lag}'] = df.groupby('Ticker')['Close'].shift(lag)
        df[f'Volume_Lag_{lag}'] = df.groupby('Ticker')['Volume'].shift(lag)
        df[f'Days_Since_Lag_{lag}'] = df.groupby('Ticker')['Date'].diff(lag).dt.days
    return df

In [24]:
def add_weighted_mean_std(df, lookback_periods):
    df = df.sort_values(['Ticker', 'Date']).copy()

    for lookback in lookback_periods:
        weighted_means = []
        weighted_stds = []

        for ticker, group in df.groupby('Ticker'):
            group = group.sort_values('Date')
            closes = group['Close'].reset_index(drop=True)
            dates = group['Date'].reset_index(drop=True)

            n = len(group)
            mean_vals = np.full(n, np.nan)
            std_vals = np.full(n, np.nan)

            for idx in range(n):
                if idx < lookback:
                    continue  # can't calculate for first points

                close_lags = closes.iloc[idx-lookback:idx]
                date_lags = dates.iloc[idx-lookback:idx]
                current_date = dates.iloc[idx]

                days_diff = (current_date - date_lags).dt.days

                if days_diff.sum() == 0:
                    continue  # avoid division by zero
                
                weight = 1/days_diff

                if weight.sum() == 0:
                    continue  # avoid division by zero

                if pd.isna(close_lags).any():
                    continue  # Skip iteration or handle NaNs in the appropriate way

                weighted_mean = (weight * close_lags).sum() / weight.sum()
                mean_vals[idx] = weighted_mean

                weighted_var = (weight * (close_lags - weighted_mean)**2).sum() / weight.sum()
                weighted_std = np.sqrt(weighted_var)
                std_vals[idx] = weighted_std

            weighted_means.append(pd.Series(mean_vals, index=group.index))
            weighted_stds.append(pd.Series(std_vals, index=group.index))

        df[f'Weighted_Mean_{lookback}'] = pd.concat(weighted_means)
        df[f'Weighted_Std_{lookback}'] = pd.concat(weighted_stds)

    return df

In [25]:
def add_simple_mean_std(df, lookback_periods):
    df = df.sort_values(['Ticker', 'Date']).copy()

    for lookback in lookback_periods:
        means = []
        stds = []

        for ticker, group in df.groupby('Ticker'):
            group = group.sort_values('Date')
            closes = group['Close_Lag_1'].reset_index(drop=True)

            mean_vals = closes.rolling(window=lookback, min_periods=lookback).mean()
            std_vals = closes.rolling(window=lookback, min_periods=lookback).std()

            means.append(pd.Series(mean_vals.values, index=group.index))
            stds.append(pd.Series(std_vals.values, index=group.index))

        df[f'Simple_Mean_{lookback}'] = pd.concat(means)
        df[f'Simple_Std_{lookback}'] = pd.concat(stds)

    return df


In [26]:
def add_returns(df):
    df['return_since_last_entry'] = df['Close']/df['Close_Lag_1'] -1

    return df

In [27]:
df = add_lags(df, [1,2,3,4,5,10,20,30,40,50,60,70,80,90,100])

df = add_weighted_mean_std(df,[1,2,3,4,5,10,20,30,40,50,60,70,80,90,100])

df = add_simple_mean_std(df,[1,2,3,4,5,10,20,30,40,50,60,70,80,90,100])

df


,Date,Dividends,Stock Splits,Ticker,Industry_Tag,Country,Volume,Close,Year,Month,...,Simple_Mean_60,Simple_Std_60,Simple_Mean_70,Simple_Std_70,Simple_Mean_80,Simple_Std_80,Simple_Mean_90,Simple_Std_90,Simple_Mean_100,Simple_Std_100
2,2000-01-11 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,441548800.0,0.702910,2000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000-01-21 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,495924800.0,0.843586,2000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2000-02-08 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,408643200.0,0.870585,2000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,2000-02-15 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,485744000.0,0.901846,2000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2000-02-24 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,376432000.0,0.873071,2000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,2021-08-19 04:00:00+00:00,0.0,0.0,ZM,technology,usa,2476000.0,332.380005,2021,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4539,2022-02-28 05:00:00+00:00,0.0,0.0,ZM,technology,usa,12033100.0,132.600006,2022,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4560,2022-03-25 04:00:00+00:00,0.0,0.0,ZM,technology,usa,4209400.0,113.930000,2022,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4575,2022-04-08 04:00:00+00:00,0.0,0.0,ZM,technology,usa,3953300.0,110.900002,2022,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df = add_returns(df)

In [29]:
pd.set_option('display.max_columns', None)
df[df['Ticker'] == 'FL'][:20]

,Date,Dividends,Stock Splits,Ticker,Industry_Tag,Country,Volume,Close,Year,Month,Day,DayOfWeek,Day_Number,Close_Lag_1,Volume_Lag_1,Days_Since_Lag_1,Close_Lag_2,Volume_Lag_2,Days_Since_Lag_2,Close_Lag_3,Volume_Lag_3,Days_Since_Lag_3,Close_Lag_4,Volume_Lag_4,Days_Since_Lag_4,Close_Lag_5,Volume_Lag_5,Days_Since_Lag_5,Close_Lag_10,Volume_Lag_10,Days_Since_Lag_10,Close_Lag_20,Volume_Lag_20,Days_Since_Lag_20,Close_Lag_30,Volume_Lag_30,Days_Since_Lag_30,Close_Lag_40,Volume_Lag_40,Days_Since_Lag_40,Close_Lag_50,Volume_Lag_50,Days_Since_Lag_50,Close_Lag_60,Volume_Lag_60,Days_Since_Lag_60,Close_Lag_70,Volume_Lag_70,Days_Since_Lag_70,Close_Lag_80,Volume_Lag_80,Days_Since_Lag_80,Close_Lag_90,Volume_Lag_90,Days_Since_Lag_90,Close_Lag_100,Volume_Lag_100,Days_Since_Lag_100,Weighted_Mean_1,Weighted_Std_1,Weighted_Mean_2,Weighted_Std_2,Weighted_Mean_3,Weighted_Std_3,Weighted_Mean_4,Weighted_Std_4,Weighted_Mean_5,Weighted_Std_5,Weighted_Mean_10,Weighted_Std_10,Weighted_Mean_20,Weighted_Std_20,Weighted_Mean_30,Weighted_Std_30,Weighted_Mean_40,Weighted_Std_40,Weighted_Mean_50,Weighted_Std_50,Weighted_Mean_60,Weighted_Std_60,Weighted_Mean_70,Weighted_Std_70,Weighted_Mean_80,Weighted_Std_80,Weighted_Mean_90,Weighted_Std_90,Weighted_Mean_100,Weighted_Std_100,Simple_Mean_1,Simple_Std_1,Simple_Mean_2,Simple_Std_2,Simple_Mean_3,Simple_Std_3,Simple_Mean_4,Simple_Std_4,Simple_Mean_5,Simple_Std_5,Simple_Mean_10,Simple_Std_10,Simple_Mean_20,Simple_Std_20,Simple_Mean_30,Simple_Std_30,Simple_Mean_40,Simple_Std_40,Simple_Mean_50,Simple_Std_50,Simple_Mean_60,Simple_Std_60,Simple_Mean_70,Simple_Std_70,Simple_Mean_80,Simple_Std_80,Simple_Mean_90,Simple_Std_90,Simple_Mean_100,Simple_Std_100,return_since_last_entry
13,2000-01-28 05:00:00+00:00,0.0,0.0,FL,footwear,usa,2029500.0,3.356755,2000,1,28,4,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2000-02-02 05:00:00+00:00,0.0,0.0,FL,footwear,usa,879500.0,3.639429,2000,2,2,2,29,3.356755,2029500.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.356755,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.356755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084210
48,2000-04-18 04:00:00+00:00,0.0,0.0,FL,footwear,usa,390400.0,6.077493,2000,4,18,1,104,3.639429,879500.0,75.0,3.356755,2029500.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.639429,0.000000e+00,3.502651,0.141263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.639429,NaN,3.498092,0.199881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.669903
52,2000-05-08 04:00:00+00:00,0.0,0.0,FL,footwear,usa,312000.0,6.890182,2000,5,8,0,124,6.077493,390400.0,20.0,3.639429,879500.0,95.0,3.356755,2029500.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.077493,0.000000e+00,5.653482,0.924111,5.327826,1.172512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.077493,NaN,4.858461,1.723972,4.357892,1.495910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.133721
57,2000-05-12 04:00:00+00:00,0.0,0.0,FL,footwear,us

In [30]:
def add_cat_signals(df, cols):
    for col in cols:
        new_col = f"{col}_signal"
        df[new_col] = (df[col] != 0).astype(int)
    return df

In [31]:
df = add_cat_signals(df, ['Dividends','Stock Splits'])
df

,Date,Dividends,Stock Splits,Ticker,Industry_Tag,Country,Volume,Close,Year,Month,Day,DayOfWeek,Day_Number,Close_Lag_1,Volume_Lag_1,Days_Since_Lag_1,Close_Lag_2,Volume_Lag_2,Days_Since_Lag_2,Close_Lag_3,Volume_Lag_3,Days_Since_Lag_3,Close_Lag_4,Volume_Lag_4,Days_Since_Lag_4,Close_Lag_5,Volume_Lag_5,Days_Since_Lag_5,Close_Lag_10,Volume_Lag_10,Days_Since_Lag_10,Close_Lag_20,Volume_Lag_20,Days_Since_Lag_20,Close_Lag_30,Volume_Lag_30,Days_Since_Lag_30,Close_Lag_40,Volume_Lag_40,Days_Since_Lag_40,Close_Lag_50,Volume_Lag_50,Days_Since_Lag_50,Close_Lag_60,Volume_Lag_60,Days_Since_Lag_60,Close_Lag_70,Volume_Lag_70,Days_Since_Lag_70,Close_Lag_80,Volume_Lag_80,Days_Since_Lag_80,Close_Lag_90,Volume_Lag_90,Days_Since_Lag_90,Close_Lag_100,Volume_Lag_100,Days_Since_Lag_100,Weighted_Mean_1,Weighted_Std_1,Weighted_Mean_2,Weighted_Std_2,Weighted_Mean_3,Weighted_Std_3,Weighted_Mean_4,Weighted_Std_4,Weighted_Mean_5,Weighted_Std_5,Weighted_Mean_10,Weighted_Std_10,Weighted_Mean_20,Weighted_Std_20,Weighted_Mean_30,Weighted_Std_30,Weighted_Mean_40,Weighted_Std_40,Weighted_Mean_50,Weighted_Std_50,Weighted_Mean_60,Weighted_Std_60,Weighted_Mean_70,Weighted_Std_70,Weighted_Mean_80,Weighted_Std_80,Weighted_Mean_90,Weighted_Std_90,Weighted_Mean_100,Weighted_Std_100,Simple_Mean_1,Simple_Std_1,Simple_Mean_2,Simple_Std_2,Simple_Mean_3,Simple_Std_3,Simple_Mean_4,Simple_Std_4,Simple_Mean_5,Simple_Std_5,Simple_Mean_10,Simple_Std_10,Simple_Mean_20,Simple_Std_20,Simple_Mean_30,Simple_Std_30,Simple_Mean_40,Simple_Std_40,Simple_Mean_50,Simple_Std_50,Simple_Mean_60,Simple_Std_60,Simple_Mean_70,Simple_Std_70,Simple_Mean_80,Simple_Std_80,Simple_Mean_90,Simple_Std_90,Simple_Mean_100,Simple_Std_100,return_since_last_entry,Dividends_signal,Stock Splits_signal
2,2000-01-11 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,441548800.0,0.702910,2000,1,11,1,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
8,2000-01-21 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,495924800.0,0.843586,2000,1,21,4,17,0.702910,441548800.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.702910,1.110223e-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.702910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200135,0,0
19,2000-02-08 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,408643200.0,0.870585,2000,2,8,1,35,0.843586,495924800.0,18.0,0.702910,441548800.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.843586,0.000000e+00,0.788539,0.068656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.843586,NaN,0.773248,0.099474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032005,0,0
22,2000-02-15 05:00:00+00:00,0.0,0.0,AAPL,technology,usa,485744000.0,0.901846,2000,2,15,1,42,0.870585,408643200.0,7.0,0.843586,495924800.0,25.0,0.702910,441548800.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.870585,0.000000e+00,0.864679,0.011161,0.842818,0.056269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.870585,NaN,0.857086,0.019091,0.805694,0.090031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [32]:
df = df.drop('Date', axis=1)

## deleted date because it is satified with day number

def get_column_types(df):
    return pd.DataFrame({
        'column': df.columns,
        'dtype': df.dtypes.values
    })
pd.set_option('display.max_rows', None)

types_table = get_column_types(df)
print(types_table)



                      column    dtype
0                  Dividends  float64
1               Stock Splits  float64
2                     Ticker   object
3               Industry_Tag   object
4                    Country   object
5                     Volume  float64
6                      Close  float64
7                       Year    int32
8                      Month    int32
9                        Day    int32
10                 DayOfWeek    int32
11                Day_Number    int64
12               Close_Lag_1  float64
13              Volume_Lag_1  float64
14          Days_Since_Lag_1  float64
15               Close_Lag_2  float64
16              Volume_Lag_2  float64
17          Days_Since_Lag_2  float64
18               Close_Lag_3  float64
19              Volume_Lag_3  float64
20          Days_Since_Lag_3  float64
21               Close_Lag_4  float64
22              Volume_Lag_4  float64
23          Days_Since_Lag_4  float64
24               Close_Lag_5  float64
25          

In [33]:
def pre_training_analysis(df, target='Close', top_n=30):
    """
    Performs pre-model exploratory analysis:
    - Correlation heatmap
    - Distribution of top correlated features
    - Pairplot of top correlated features
    
    Parameters:
    - df (pd.DataFrame): Input DataFrame
    - target (str): Target column to predict
    - top_n (int): Number of top correlated features to visualize
    """
    
    # 1. Filter numerical columns
    numeric_df = df.select_dtypes(include=['float64', 'int32', 'int64'])
    
    # 2. Compute correlations
    corr_matrix = numeric_df.corr()
    target_corr = corr_matrix[target].drop(target).sort_values(ascending=False)

    print(f"\n🔍 Top {top_n} features most correlated with '{target}':\n")
    print(target_corr.head(top_n))

    # 3. Correlation heatmap
    plt.figure(figsize=(10, 10))
    sns.heatmap(corr_matrix[[target]].sort_values(by=target, ascending=False), annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title(f"Correlation of Numerical Features with '{target}'", fontsize=14)
    plt.show()
    
    # 4. Distribution plots
    top_features = target_corr.head(top_n).index.tolist()
    for feature in top_features:
        plt.figure(figsize=(8, 4))
        sns.scatterplot(data=df, x=feature, y=target, alpha=0.5)
        plt.title(f"Scatterplot: {feature} vs {target}")
        plt.show()
    
    # 5. Pairplot for top features
    pairplot_features = top_features[:min(5, len(top_features))] + [target]
    sns.pairplot(df[pairplot_features], diag_kind="kde", corner=True)
    plt.suptitle(f"Pairplot of Top {top_n} Correlated Features with '{target}'", y=1.02)
    plt.show()


In [40]:
# pre_training_analysis(df)
df.isna().sum()

Dividends                     0
Stock Splits                  0
Ticker                        0
Industry_Tag                  0
Country                       0
Volume                       34
Close                         0
Year                          0
Month                         0
Day                           0
DayOfWeek                     0
Day_Number                    0
Close_Lag_1                  61
Volume_Lag_1                 94
Days_Since_Lag_1             61
Close_Lag_2                 122
Volume_Lag_2                154
Days_Since_Lag_2            122
Close_Lag_3                 183
Volume_Lag_3                214
Days_Since_Lag_3            183
Close_Lag_4                 244
Volume_Lag_4                275
Days_Since_Lag_4            244
Close_Lag_5                 305
Volume_Lag_5                335
Days_Since_Lag_5            305
Close_Lag_10                610
Volume_Lag_10               638
Days_Since_Lag_10           610
Close_Lag_20               1179
Volume_L

In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression

def pre_training_analysis_v2(df, target='Close', top_n=6, drop_low_variance=True, variance_threshold=0.01):
    """
    Extended EDA for regression:
    - Drops low-variance features
    - Correlation + Mutual Info
    - Histograms, KDEs
    - Pairplots
    - Scatterplots
    
    Parameters:
    - df (pd.DataFrame): Data
    - target (str): Target column
    - top_n (int): Number of top features to show
    - drop_low_variance (bool): Drop near-constant columns
    - variance_threshold (float): Threshold for variance filter
    """
    
    # 1. Numerical subset
    numeric_df = df.select_dtypes(include=['float64', 'int32', 'int64']).copy()
    
    # 2. Drop low-variance
    if drop_low_variance:
        features = numeric_df.drop(columns=[target], errors='ignore')
        selector = VarianceThreshold(threshold=variance_threshold)
        selector.fit(features)
        retained_columns = features.columns[selector.get_support()]
        dropped_columns = features.columns[~selector.get_support()]
        
        print(f"\n⚠️ Dropping {len(dropped_columns)} low-variance features:\n{dropped_columns.tolist()}\n")
        numeric_df = numeric_df[retained_columns.tolist() + [target]]
    
    # 3. Correlation
    corr_matrix = numeric_df.corr()
    target_corr = corr_matrix[target].drop(target).sort_values(ascending=False)
    
    print(f"\n📈 Top {top_n} features by Pearson correlation with '{target}':\n")
    print(target_corr.head(top_n))
    
    # 4. Mutual Information
    mi_df = numeric_df.iloc[101:]
    X = mi_df.drop(columns=[target])
    y = mi_df[target]
    mi_scores = mutual_info_regression(X, y, random_state=42)
    mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
    
    print(f"\n🧠 Top {top_n} features by Mutual Information with '{target}':\n")
    print(mi_series.head(top_n))

    # --- PLOTS ---
    # 5. Correlation heatmap
    plt.figure(figsize=(10, 10))
    sns.heatmap(corr_matrix[[target]].sort_values(by=target, ascending=False), annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title(f"Correlation with '{target}'", fontsize=14)
    plt.show()
    
    # 6. KDE + Histograms of top mutual info features
    top_features = mi_series.head(top_n).index.tolist()
    for feature in top_features:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
        sns.histplot(df[feature], kde=False, ax=ax[0], color='skyblue')
        ax[0].set_title(f"Histogram of {feature}")
        sns.kdeplot(df[feature], ax=ax[1], fill=True, color='orange')
        ax[1].set_title(f"KDE of {feature}")
        plt.show()

    # 7. Scatterplots against target
    for feature in top_features:
        plt.figure(figsize=(8, 4))
        sns.scatterplot(x=df[feature], y=df[target], alpha=0.5)
        plt.title(f"Scatterplot: {feature} vs {target}")
        plt.show()
    
    # 8. Pairplot of top 5 MI features + target
    pairplot_features = top_features[:min(5, len(top_features))] + [target]
    sns.pairplot(df[pairplot_features], diag_kind="kde", corner=True)
    plt.suptitle(f"Pairplot of Top {top_n} MI Features + '{target}'", y=1.02)
    plt.show()


In [39]:
df.describe()

,Dividends,Stock Splits,Volume,Close,Year,Month,Day,DayOfWeek,Day_Number,Close_Lag_1,Volume_Lag_1,Days_Since_Lag_1,Close_Lag_2,Volume_Lag_2,Days_Since_Lag_2,Close_Lag_3,Volume_Lag_3,Days_Since_Lag_3,Close_Lag_4,Volume_Lag_4,Days_Since_Lag_4,Close_Lag_5,Volume_Lag_5,Days_Since_Lag_5,Close_Lag_10,Volume_Lag_10,Days_Since_Lag_10,Close_Lag_20,Volume_Lag_20,Days_Since_Lag_20,Close_Lag_30,Volume_Lag_30,Days_Since_Lag_30,Close_Lag_40,Volume_Lag_40,Days_Since_Lag_40,Close_Lag_50,Volume_Lag_50,Days_Since_Lag_50,Close_Lag_60,Volume_Lag_60,Days_Since_Lag_60,Close_Lag_70,Volume_Lag_70,Days_Since_Lag_70,Close_Lag_80,Volume_Lag_80,Days_Since_Lag_80,Close_Lag_90,Volume_Lag_90,Days_Since_Lag_90,Close_Lag_100,Volume_Lag_100,Days_Since_Lag_100,Weighted_Mean_1,Weighted_Std_1,Weighted_Mean_2,Weighted_Std_2,Weighted_Mean_3,Weighted_Std_3,Weighted_Mean_4,Weighted_Std_4,Weighted_Mean_5,Weighted_Std_5,Weighted_Mean_10,Weighted_Std_10,Weighted_Mean_20,Weighted_Std_20,Weighted_Mean_30,Weighted_Std_30,Weighted_Mean_40,Weighted_Std_40,Weighted_Mean_50,Weighted_Std_50,Weighted_Mean_60,Weighted_Std_60,Weighted_Mean_70,Weighted_Std_70,Weighted_Mean_80,Weighted_Std_80,Weighted_Mean_90,Weighted_Std_90,Weighted_Mean_100,Weighted_Std_100,Simple_Mean_1,Simple_Std_1,Simple_Mean_2,Simple_Std_2,Simple_Mean_3,Simple_Std_3,Simple_Mean_4,Simple_Std_4,Simple_Mean_5,Simple_Std_5,Simple_Mean_10,Simple_Std_10,Simple_Mean_20,Simple_Std_20,Simple_Mean_30,Simple_Std_30,Simple_Mean_40,Simple_Std_40,Simple_Mean_50,Simple_Std_50,Simple_Mean_60,Simple_Std_60,Simple_Mean_70,Simple_Std_70,Simple_Mean_80,Simple_Std_80,Simple_Mean_90,Simple_Std_90,Simple_Mean_100,Simple_Std_100,return_since_last_entry,Dividends_signal,Stock Splits_signal
count,4988.000000,4988.000000,4.954000e+03,4988.000000,4988.000000,4988.000000,4988.000000,4988.000000,4988.000000,4927.000000,4.894000e+03,4927.000000,4866.000000,4.834000e+03,4866.000000,4805.000000,4.774000e+03,4805.000000,4744.000000,4.713000e+03,4744.000000,4683.000000,4.653000e+03,4683.000000,4378.000000,4.350000e+03,4378.000000,3809.000000,3.782000e+03,3809.000000,3281.000000,3.259000e+03,3281.000000,2776.000000,2.758000e+03,2776.000000,2284.000000,2.268000e+03,2284.000000,1804.000000,1.790000e+03,1804.000000,1347.000000,1.336000e+03,1347.000000,941.000000,9.330000e+02,941.000000,571.000000,5.680000e+02,571.000000,269.000000,2.680000e+02,269.000000,4927.000000,4.927000e+03,4866.000000,4866.000000,4805.000000,4805.000000,4744.000000,4744.000000,4683.000000,4683.000000,4378.000000,4378.000000,3809.000000,3809.000000,3281.000000,3281.000000,2776.000000,2776.000000,2284.000000,2284.000000,1804.000000,1804.000000,1347.000000,1347.000000,941.000000,941.000000,571.000000,571.000000,269.000000,269.000000,4927.000000,0.0,4866.000000,4866.000000,4805.000000,4805.000000,4744.000000,4744.000000,4683.000000,4683.000000,4378.000000,4378.000000,3809.000000,3809.000000,3281.000000,3281.000000,2776.000000,2776.000000,2284.000000,2284.000000,1804.000000,1804.000000,1347.000000,1347.000000,941.000000,941.000000,571.000000,571.000000,269.000000,269.000000,4927.000000,4988.000000,4988.000000
mean,0.003335,0.000802,2.496811e+07,64.897845,2012.644948,6.589816,15.745589,2.043304,4799.460505,63.631735,2.513081e+07,79.961031,62.431113,2.528115e+07,159.935676,61.319324,2.543831e+07,239.961498,60.217532,2.559279e+07,320.089165,59.003507,2.575156e+07,400.577408,51.672824,2.662563e+07,805.138419,40.081930,2.836035e+07,1613.729325,32.619696,3.059129e+07,2418.785736,26.633306,3.234039e+07,3221.103026,22.661064,3.400579e+07,4005.092382,20.516583,3.652789e+07,4774.136918,19.266628,3.298889e+07,5547.867112,18.007011,2.661390e+07,6327.043571,17.139025,2.843556e+07,7030.332750,17.486859,3.461833e+07,7646.237918,63.631735,7.500258e-16,63.719212,2.649434,63.830821,4.091064,63.945612,5.171265,64.080533,6.032730,64.824791,9.007243,67.243117,13.229757,70.489010,16.934719,74.533773,20.498284,79.383694,24.043436,81.225508,26.331081,88.036679,30.566554,98.844886,36.801718,113.386454,44.

In [37]:
pre_training_analysis_v2(df, top_n=10, drop_low_variance=True)

c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\feature_selection\_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)



⚠️ Dropping 6 low-variance features:
['Dividends', 'Stock Splits', 'Weighted_Std_1', 'Simple_Std_1', 'Dividends_signal', 'Stock Splits_signal']


📈 Top 10 features by Pearson correlation with 'Close':

Simple_Mean_1       0.981482
Close_Lag_1         0.981482
Weighted_Mean_1     0.981482
Weighted_Mean_2     0.979466
Weighted_Mean_3     0.976961
Weighted_Mean_4     0.975266
Simple_Mean_2       0.975072
Weighted_Mean_5     0.973934
Weighted_Mean_10    0.970071
Weighted_Mean_20    0.968629
Name: Close, dtype: float64


ValueError: Input X contains NaN.

In [ ]:
# Numerical features
numerical_features = df.select_dtypes(['float64','int64']).columns.tolist()
numerical_features.remove("Close")


# Categorical features
categorical_features = df.select_dtypes(['object','int32']).columns.tolist()

print(categorical_features)

['Ticker', 'Industry_Tag', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Dividends_signal', 'Stock Splits_signal']


In [ ]:
def split_time_series_data(df, split_percentage=0.7):
    """
    Splits a time series dataset into training and testing sets by ticker, based on the given percentage.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the time series data with 'date' and 'ticker' columns.
    split_percentage (float): The percentage of data to use for the training set (e.g., 0.7 for 70% training).
    
    Returns:
    X_train, X_test, y_train, y_test: Split datasets for features and target.
    """

    # Sort the DataFrame by 'ticker' and 'date' to maintain chronological order
    df = df.sort_values(by=['Ticker', 'Day_Number'])

    # Create empty lists to hold training and test data
    train_data = []
    test_data = []

    # Loop through each unique ticker
    for ticker in df['Ticker'].unique():
        # Filter data for this ticker
        ticker_data = df[df['Ticker'] == ticker]
        
        # Calculate the index that separates the split based on the given percentage
        split_idx = int(len(ticker_data) * split_percentage)
        
        # Split the data into train and test sets
        train_data.append(ticker_data.iloc[:split_idx])  # First 'split_percentage' for training
        test_data.append(ticker_data.iloc[split_idx:])  # Remaining data for testing

    # Concatenate the train and test data
    train_df = pd.concat(train_data).sort_values(by=['Ticker', 'Day_Number'])
    test_df = pd.concat(test_data).sort_values(by=['Ticker', 'Day_Number'])

    # Assuming 'target' is the name of your target column, adjust as needed
    X_train = train_df.drop(columns='Close')  # Features for training
    y_train = train_df['Close']  # Target for training

    X_test = test_df.drop(columns='Close')  # Features for testing
    y_test = test_df['Close']  # Target for testing

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_time_series_data(df, split_percentage=0.7)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (3464, 34)
X_test shape: (1524, 34)
y_train shape: (3464,)
y_test shape: (1524,)


In [ ]:

numeric_preprocessor = Pipeline([ 
    ("scaler", StandardScaler())
])                  ###might be wrong becaue i scale over all tickes

categorical_preprocessor = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False, drop='first',handle_unknown='infrequent_if_exist'))
])

preprocessor = ColumnTransformer([
    ("numerical",numeric_preprocessor, numerical_features),
    ("categorical",categorical_preprocessor, categorical_features)
    
], remainder='passthrough')

from xgboost import XGBRegressor

pipe = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',  XGBRegressor(random_state=0))
])

from scipy.stats import uniform as sp_uniform

param_grid = {
    'regressor__learning_rate': sp_uniform(loc=0.03, scale=0.07),
    'regressor__max_depth': [4, 5, 6, 7],
    'regressor__n_estimators': [300,400,500,600,700,800,900,1000]
}

grid_search = RandomizedSearchCV(pipe, param_distributions=param_grid, cv=5, random_state=0, n_jobs=-1,n_iter=15,error_score='raise' )
grid_search.fit(X_train, y_train)



print(pipe.get_params())
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print('Best parameters:',best_params)

c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


{'memory': None, 'steps': [('preprocessor', ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['Dividends', 'Stock Splits', 'Volume',
                                  'Day_Number', 'Close_Lag_1', 'Volume_Lag_1',
                                  'Days_Since_Lag_1', 'Close_Lag_2',
                                  'Volume_Lag_2', 'Days_Since_Lag_2',
                                  'Close_Lag_3', 'Volume_Lag_3',
                                  'Days_Since_Lag_3', 'Weighted_Mean_3',
                                  'Weighted_Std_3', 'Weight...
                                  'Weighted_Std_100', 'Simple_Mean_3',
                                  'Simple_Std_3', 'Simple_Mean_50',
                                  'Simple_Std_50', 'Simple_Mean_100',
                                  'Simple_Std_100']),
                               

c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Best parameters: {'regressor__learning_rate': 0.038279209810825325, 'regressor__max_depth': 4, 'regressor__n_estimators': 400}
